In [1]:
import pandas as pd
import numpy as np
import requests
import json

from config import gkey

In [3]:
ethnic = {"ethnicity": ["chinese", "cuban", "czech", "french", "german"]}
types_df = pd.DataFrame(ethnic)

In [4]:
# set up additional columns to hold information
types_df['name'] = ""
types_df['address'] = ""
types_df['price_level'] = ""
types_df['rating'] = ""
types_df

,ethnicity,name,address,price_level,rating
0,chinese,,,,
1,cuban,,,,
2,czech,,,,
3,french,,,,
4,german,,,,


In [5]:
# find the closest restaurant of each type to coordinates

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "location": "39.952583,-75.16522",  # philadelphia coords,
    "rankby": "distance",
    "type": "restaurant",
    "key": gkey,
}
# use iterrows to iterate through pandas dataframe
for index, row in types_df.iterrows():

    # get restaurant type from df
    restr_type = row['ethnicity']

    # add keyword to params dict
    params['keyword'] = restr_type

    # assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest {restr_type} restaurant is {results[0]['name']}.")
        
        types_df.loc[index, 'name'] = results[0]['name']
        types_df.loc[index, 'address'] = results[0]['vicinity']
        types_df.loc[index, 'price_level'] = results[0]['price_level']
        types_df.loc[index, 'rating'] = results[0]['rating']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
types_df

Closest chinese restaurant is Chinese Fast Wok.
------------
Closest cuban restaurant is Bar Bombón.
------------
Closest czech restaurant is Balkan Express.
------------
Closest french restaurant is Amuse.
------------
Closest german restaurant is Tir na nOg Irish Bar & Grill.
------------


,ethnicity,name,address,price_level,rating
0,chinese,Chinese Fast Wok,"1500 John F Kennedy Blvd, Philadelphia",2,2.3
1,cuban,Bar Bombón,"133 S 18th St, Philadelphia",2,4.6
2,czech,Balkan Express,"2237 Grays Ferry Ave, Philadelphia",2,4.7
3,french,Amuse,"1421 Arch St, Philadelphia",4,4.1
4,german,Tir na nOg Irish Bar & Grill,"1600 Arch St, Philadelphia",2,4.2


### Another example

In [7]:
address_data = {
    "City": ["New York City", "Los Angeles", "Chicago", "Houston", "Philadelphia"],
    "State": ["New York", "California", "Illinois","Texas", "Pennsylvania"]
}

cities_pd = pd.DataFrame(address_data)
# Add columns for lat, lng, airport name, airport address, airport rating
# Note that we used "" to specify initial entry.
cities_pd["Lat"] = ""
cities_pd["Lng"] = ""
cities_pd["Airport Name"] = ""
cities_pd["Airport Address"] = ""
cities_pd["Airport Rating"] = ""
cities_pd

,City,State,Lat,Lng,Airport Name,Airport Address,Airport Rating
0,New York City,New York,,,,,
1,Los Angeles,California,,,,,
2,Chicago,Illinois,,,,,
3,Houston,Texas,,,,,
4,Philadelphia,Pennsylvania,,,,,


In [8]:
# create a params dict that will be updated with new city each iteration
params = {"key": gkey}

# Loop through the cities_pd and run a lat/long search for each city
for index, row in cities_pd.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    city = row['City']
    state = row['State']

    # update address key value
    params['address'] = f"{city},{state}"

    # make request
    cities_lat_lng = requests.get(base_url, params=params)
    
    # convert to json
    cities_lat_lng = cities_lat_lng.json()

    cities_pd.loc[index, "Lat"] = cities_lat_lng["results"][0]["geometry"]["location"]["lat"]
    cities_pd.loc[index, "Lng"] = cities_lat_lng["results"][0]["geometry"]["location"]["lng"]

# Visualize to confirm lat lng appear
cities_pd.head()

,City,State,Lat,Lng,Airport Name,Airport Address,Airport Rating
0,New York City,New York,40.712775,-74.005973,,,
1,Los Angeles,California,34.052234,-118.243685,,,
2,Chicago,Illinois,41.878114,-87.629798,,,
3,Houston,Texas,29.760427,-95.369803,,,
4,Philadelphia,Pennsylvania,39.952584,-75.165222,,,


In [9]:
# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "airport",
    "keyword": "international airport",
    "key": gkey
}

# Use the lat/lng we recovered to identify airports
for index, row in cities_pd.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        cities_pd.loc[index, "Airport Name"] = name_address["results"][0]["name"]
        cities_pd.loc[index, "Airport Address"] = name_address["results"][0]["vicinity"]
        cities_pd.loc[index, "Airport Rating"] = name_address["results"][0]["rating"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

cities_pd

,City,State,Lat,Lng,Airport Name,Airport Address,Airport Rating
0,New York City,New York,40.712775,-74.005973,John F. Kennedy International Airport,Queens,3.9
1,Los Angeles,California,34.052234,-118.243685,Los Angeles International Airport,"1 World Way, Los Angeles",3.8
2,Chicago,Illinois,41.878114,-87.629798,O'Hare International Airport,"10000 W O'Hare Ave, Chicago",3.9
3,Houston,Texas,29.760427,-95.369803,George Bush Intercontinental Airport,"2800 N Terminal Rd, Houston",4.1
4,Philadelphia,Pennsylvania,39.952584,-75.165222,Philadelphia International Airport,"8000 Essington Ave, Philadelphia",3.7
